Before we get the data and start exploring it, let's download all the dependencies that we will need.

Also keep in mind that in the code on means toronto n means neighborhood v means venues df is the first dff second and dfff is the merged dataframe

on = toronto
on_n=toronto neighbourhood
on_v = toronto venues 


In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


1. Download and Explore Dataset

In [26]:
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print('Data downloaded!')
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

Data downloaded!


Tranform the data into a pandas dataframe
The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [27]:
# creating a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
on = pd.DataFrame(columns = column_names)

Data cleaning by elimaneting unnecessary charachters in the postcode column this step is important because later on we are going to merge two datasets by looking the postcodes if two dataframes same neighbourhoods postcodes are not identical it is not going to work

In [31]:
content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    on = on.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
on = on[on.Borough!=' Not assigned ']
on = on[on.Borough!= 0]
on.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,on.shape[0]):
    if on.iloc[i][2] == 'Not assigned':
        on.iloc[i][2] = on.iloc[i][1]
        i = i+1
                                 
df = on.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df[1:11]

,Postalcode,Borough,Neighborhood
1,M1B\n,Scarborough\n,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill, Guildwood, ..."
4,M1G\n,Scarborough\n,"Woburn, Woburn, Woburn, Woburn"
5,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae"
6,M1J\n,Scarborough\n,"Scarborough Village, Scarborough Village, Scar..."
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park, K..."
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge, Golden Mile, ..."
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village Wes..."
10,M1N\n,Scarborough\n,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


In [32]:
print('The dataframe has {} boroughs and {} Neighborhood.'.format(
        len(on['Borough'].unique()),
        on.shape[0]
    )
)

The dataframe has 11 boroughs and 720 Neighborhood.


Below you are going to see the taken coordinated data 

In [41]:
on_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' on_geocsv
geocsv_data = pd.read_csv(on_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


One cleaning for second dataset and after that we are going to merge these together with pd 

In [42]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        dff.loc[len(df)] = row_data
        dff.head()

merging..

In [43]:
dfff = pd.merge(geocsv_data, dff, on="Postal Code",)
dfff.head()
dfff

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
2,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
3,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
...,...,...,...,...,...
200,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
201,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
202,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
203,M9W,43.706748,-79.594054,Etobicoke,"Northwest, Northwest"


In [51]:

dfff = dfff[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
dfff.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
4,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
6,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
7,M1G,Scarborough,Woburn,43.770992,-79.216917
8,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [62]:
dfff.to_csv('on.csv',index=False)

In [63]:
print('The dataframe has {} boroughs and {} Neighborhood.'.format(
        len(on['Borough'].unique()),
        dfff.shape[0]
    )
)

The dataframe has 11 boroughs and 205 Neighborhood.


In [64]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [74]:
# creating a map of toronto using latitude and longitude values
map_on = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfff['Latitude'], dfff['Longitude'], dfff['Borough'], dfff['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_on)  
map_on


In [66]:
on_csv = 'on.csv'

In [67]:
on_n = pd.read_csv(on_csv)

In [68]:
on_n.shape
on_n.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
4,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711


In [103]:
CLIENT_ID = "0RHQV0FBMRNOWZVVEAOI5CLSTZY3JJO2P2HFVHDCQVHOH4AM" # your Foursquare ID
CLIENT_SECRET = 'X05TQTCX31WKBYEC3YHMY2AXIF3GVPUOXJXVIRBK5VDYRVK4'# your Foursquare Secret
VERSION = '20180604'
LIMIT = 30


In [88]:
dfff.loc[0, 'Neighborhood']

'Malvern, Rouge, Malvern, Rouge'

In [96]:
neighborhood_latitude = dfff.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfff.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfff.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge, Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
on_v = getNearbyVenues(names=on_n['Neighborhood'],
                                   latitudes=on_n['Latitude'],
                                   longitudes=on_n['Longitude']
                                  )

Malvern, Rouge, Malvern, Rouge
Malvern, Rouge
Rouge Hill, Port Union, Highland Creek, Rouge Hill, Port Union, Highland Creek
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill, Guildwood, Morningside, West Hill
Guildwood, Morningside, West Hill
Woburn, Woburn
Woburn
Cedarbrae, Cedarbrae
Cedarbrae
Scarborough Village, Scarborough Village
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park, Kennedy Park, Ionview, East Birchmount Park
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge, Golden Mile, Clairlea, Oakridge
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West, Cliffside, Cliffcrest, Scarborough Village West
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West, Birch Cliff, Cliffside West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre, Dorset Park, Wexford Heights, Scarborough Town Centre
Dorset Park, Wexford Heights, Scarborough Town Cen

In [101]:
print(on_v.shape)
on_v.head()

(410, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
3,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
4,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
